In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import datetime

import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import tqdm

import mtglobals

In [3]:
client, mt_env = mtglobals.gen_client()

Your account balance is 498226.73


In [4]:
# Get the list of HITs you want to compile
all_hits = mtglobals.download_all_hits(client, start_cutoff=datetime.datetime(2021,10,1))

p2:VaEOH9tTNqVc9GdKvouhb8gfApWXnQUg6L8vX7Ep1kgOrCLEjW9/PNnYFfs3fc8=
p2:AgrE9bUk8gMeHc9y7Ilr1H62rKiJvjVr5sXYGd1dD2W7AKZbJGFQGcj69ayxn6k=
p2:c4ioMFZTOQBg3fiJRfELPLJ1jNlQ9broecjHapJabDSAxcAdFaicTVjc/Q5lEJg=
p2:utz0tiNNqcaDDFjhBTdaXoHv6PD03yl91QRnyTUomEUAmVC4MHDPT/1gdVE+peQ=
p2:cqAQcC4Ql7wCtVBM+lLF7fMwcf+THBHE5QfBdip+S3H8KTDUlMOHlt1i9s3NKOw=
p2:GFbb5bPIMf6aLGjg+I9kOzgp934OvlPFvQ+R6vJyu91dbsbHqnZh6sTeyViCEjo=
p2:7vKS4hpOXekhDXPy1nAyJN0qfFWdikQ6cuaQDXcE7Pv+vvA/lFuuolnhO7Fl/vU=
p2:PKI3R7PYZP69ARXhq1eR2WWCxbrkAdBys3Y1go2FW1Ous29mniDpeoaE8GiBn9Q=
p2:izLgvXXWrFkQRErZm4EuSkB6/VyvUji6BQs20mqg2SCMOe/8AOdkX93SsVCYXqEa
p2:UN7psWIxYedfR8ssUf5FhkoX8rPJqzoEJpSorswCJesxi8Aj9NUOZiJKc9scB40r
p2:wLBjWAlVqQapL2fwMxCjU028ZvwgWfyyaFfVnQPliHtTtiQveRp7Ful3btAP0SPt
p2:0PtoZLs/FzG09dcQKWaz0ndAIYu08a8SwGewVi1zrieUlpwDQxfz6tDlr5fumVF/
p2:87kGqzhHuU/3kjumBX0Jtb5ge/A7FzX47ZhMSNlYrz+I8Qs2QUxW0nOxlkP3tV1O
p2:8/L9+hZBc6lWzfuPG7udJPnIwR6NHF8zA+nIByD/CURRJkDMD4ZFUyz3VtCe9iCI
p2:zgE8IQaf11Xjo/uN0GYM0emA771lc8eXplMMtK+JDLuRp

In [5]:
len(all_hits)

2463

In [6]:
 qual_names = ['Workplace_Survey_Custom06','Workplace_Survey_Custom07',
               'Workplace_Survey_Custom08','Workplace_Survey_Custom09',
               'Workplace_Survey_Custom10','Workplace_Survey_Custom11',
               'Workplace_Survey_Custom12','Workplace_Survey_Custom13',]
#qual_names = ['Workplace_Survey_Custom12']

In [7]:
all_worker_ids = []
for cur_qual_name in qual_names:
    worker_ids = mtglobals.get_workers_with_qual(client, cur_qual_name)
    all_worker_ids.extend(worker_ids)

In [8]:
len(all_worker_ids)

2016

Get the HITs for each of these workers

In [9]:
# (worker_id, hit) tuples
worker_tuples = [(wid, mtglobals.get_hit_for_worker(all_hits, wid)) for wid in all_worker_ids]

In [10]:
all_submit_info = []
for cur_tuple in tqdm.tqdm(worker_tuples):
    worker_id = cur_tuple[0]
    cur_hit = cur_tuple[1]
    hit_id = cur_hit['HITId']
    reward = cur_hit['Reward']
    num_avail = cur_hit['NumberOfAssignmentsAvailable']
    creation = cur_hit['CreationTime']
    # Convert to non-tz
    creation_naive = creation.replace(tzinfo=None)
    # 1 - this is a 0/1 accepted vs. not accepted
    accepted = 1 - num_avail
    #print(f"Processing {hit_id}, {reward}")
    submit_info = {'worker_id':worker_id, 'wage':reward, 'accepted':accepted,
                   'stage2_creation':creation_naive, 'stage2_creation_str':str(creation)}
    all_submit_info.append(submit_info)

100%|███████████████████████████████████████████████████████████████████████████| 2016/2016 [00:00<00:00, 91645.99it/s]


In [11]:
#cur_hit

In [12]:
all_submit_info[0]

{'worker_id': 'A1AHWC9TUBF2Z4',
 'wage': '1.50',
 'accepted': 0,
 'stage2_creation': datetime.datetime(2021, 12, 8, 19, 16, 16),
 'stage2_creation_str': '2021-12-08 19:16:16-08:00'}

In [13]:
# Transform this into a .dta for regression
accepted_df = pd.DataFrame(all_submit_info)

In [14]:
accepted_df['lwage'] = accepted_df['wage'].apply(lambda x: np.log(float(x)))

In [15]:
timestamp = str(datetime.datetime.now()).split(".")[0].replace(" ","_").replace("-","").replace(":","")

In [16]:
reg_result = smf.ols(formula='accepted ~ lwage', data=accepted_df).fit(cov_type='HC1')
print(reg_result.summary())

                            OLS Regression Results                            
Dep. Variable:               accepted   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                     12.08
Date:                Sun, 23 Jan 2022   Prob (F-statistic):           0.000522
Time:                        13:25:55   Log-Likelihood:                -1295.0
No. Observations:                2016   AIC:                             2594.
Df Residuals:                    2014   BIC:                             2605.
Df Model:                           1                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.6965      0.010     67.908      0.0

In [17]:
# Also get the sample mean of the dependent var
np.mean(accepted_df['accepted'])

0.6924603174603174

### Check who is suspended

In [18]:
wids = list(accepted_df['worker_id'].values)

In [19]:
with open('./mt_log.txt', 'r', encoding='utf-8') as f:
    mt_log = [l.strip().lower() for l in f.readlines()]

In [20]:
def get_wid_lines(worker_id):
    wid_lines = [line for line in mt_log if worker_id.lower() in line]
    return wid_lines
wid_lines = {wid: get_wid_lines(wid) for wid in wids}

In [21]:
failures = []
for cur_wid, cur_lines in wid_lines.items():
    fail = [l for l in cur_lines if "fail" in l]
    if len(fail) > 0:
        failures.append((cur_wid, fail))

In [22]:
fail_wids = [t[0] for t in failures]

In [23]:
fail_all = {k:v for k, v in wid_lines.items() if k in fail_wids}

In [24]:
len(fail_all)

51

In [25]:
# And merge

In [26]:
accepted_df['suspended'] = accepted_df['worker_id'].apply(lambda x: True if x in fail_wids else False)

In [27]:
accepted_df['suspended'].value_counts()

False    1965
True       51
Name: suspended, dtype: int64

In [28]:
output_fpath = f'../results_2stage/accepted_df_{timestamp}.dta'
accepted_df.to_stata(output_fpath)